In [15]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import random
import gc

### Revision de estadisticas de la base (manzanas)

In [2]:
amy_manz= gpd.read_file('og_base\manzanas_caracterizadas_lima.shp')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\gonza\AppData\Local\Temp\ipykernel_20876\1518693472.py:1: SyntaxWarning: invalid escape sequence '\m'
  amy_manz= gpd.read_file('og_base\manzanas_caracterizadas_lima.shp')


In [3]:
amy_manz.NIV_VULNE.value_counts()

NIV_VULNE
ALTA         55529
MUY ALTA     22671
MEDIA        16580
SIN VULNE     7476
BAJA          4714
Name: count, dtype: int64

In [4]:
filt_manz = amy_manz[(amy_manz.NIV_VULNE=='MUY ALTA') | (amy_manz.NIV_VULNE=='BAJA')]

In [6]:
filt_manz.to_file('manzanas_filtradas_lima')

### Tratamiento de imagenes

Extrajimos las mascaras a traves de QGIS, del satelite de google earth

El procesamiento a seguir para todas las imagenes es:
- Reescalado de imagenes 
- Almacenamiento de las matrices en objetos .npy

El procesamiento de las mascaras es:
- Limpieza de clases (imagenes almacenadas en cmyk, transformacion a rgb y matrices con las clases por pixel)

#### Limpieza de clases en las mascaras

In [6]:
#Mis mascaras estan en:
mascaras_dir = "Input\Mascaras"
solucion_dir = "Input\puedo_arreglarlo"
masc_par=[os.path.join(mascaras_dir,i) for i in os.listdir(mascaras_dir)]

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:3: SyntaxWarning: invalid escape sequence '\p'
C:\Users\gonza\AppData\Local\Temp\ipykernel_9844\811419825.py:2: SyntaxWarning: invalid escape sequence '\M'
  mascaras_dir = "Input\Mascaras"
C:\Users\gonza\AppData\Local\Temp\ipykernel_9844\811419825.py:3: SyntaxWarning: invalid escape sequence '\p'
  solucion_dir = "Input\puedo_arreglarlo"


In [ ]:
# Definimos los colores
green = np.array([0, 255, 0])   # Clase 1
red = np.array([255, 0, 0])     # Clase 2
blue = np.array([0, 0, 255])     # Clase 3
white = np.array([255, 255, 255])     # Clase 0 (fondo)
black = np.array([0, 0, 0])     # Clase 0 (fondo)
tolerance = 50

def correct_and_convert_mask(file_path, output_dir):
    """
    Corrige y convierte una máscara RGB a un canal de índices de clase.
    Guarda directamente el resultado en el disco.
    """
    # Cargamos la imagen
    mask_rgb = np.array(Image.open(file_path).convert('RGB'))
    
    # Crear una máscara booleana para cada color con tolerancia
    green_mask = np.all(np.abs(mask_rgb - green) <= tolerance, axis=-1)
    red_mask = np.all(np.abs(mask_rgb - red) <= tolerance, axis=-1)
    black_mask = np.all(np.abs(mask_rgb - black) <= tolerance, axis=-1)
    blue_mask = np.all(np.abs(mask_rgb - blue) <= tolerance, axis=-1)
    white_mask = np.all(np.abs(mask_rgb - white) <= tolerance, axis=-1)
    
    # Inicializar la máscara de un solo canal
    mask_class = np.zeros((mask_rgb.shape[0], mask_rgb.shape[1]), dtype=np.uint8)
    
    # Asignar los índices de clase según las máscaras booleanas
    mask_class[green_mask] = 1
    mask_class[red_mask] = 2
    mask_class[blue_mask] = 3
    mask_class[black_mask] = 0
    mask_class[white_mask] = 0

    # Guardar la máscara procesada
    output_path = os.path.join(output_dir, os.path.basename(file_path))
    Image.fromarray(mask_class).save(output_path)
    
    # Liberar memoria
    del mask_rgb, green_mask, red_mask, black_mask, mask_class
    gc.collect()

# Procesamiento en paralelo usando hilos
def process_images(muestra, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    with ThreadPoolExecutor(max_workers=4) as executor:
        # Procesar y guardar directamente
        list(tqdm(executor.map(lambda file: correct_and_convert_mask(file, output_dir), muestra), total=len(muestra)))

# Subconjunto de imágenes a procesar (ya hice las primeras 7000 [:7000], luego otros 9000 [7000:16000])
masc_par_subset = masc_par[16000:]


# Procesar las imágenes
process_images(masc_par_subset, solucion_dir)


100%|██████████| 2779/2779 [23:49<00:00,  1.94it/s]


#### Reescalado de imagenes

In [41]:
images_path = r'Input\Masc_Sat'
masks_path = r'Input\puedo_arreglarlo'

In [42]:
def process_image(file_name):
    sat_path = os.path.join(images_path, file_name)
    mask_path = os.path.join(masks_path, file_name)
    
    np_sat = np.array(Image.open(sat_path).resize((256, 256)))[..., :3]
    np_mask = np.array(Image.open(mask_path).resize((256, 256)))
    return np_sat, np_mask

with ThreadPoolExecutor(max_workers=5) as executor:
    file_names = os.listdir(images_path)
    
    results = list(tqdm(executor.map(process_image, file_names), total=len(file_names)))

# Separar los resultados en sat_recal y mask_recal
sat_recal, mask_recal = zip(*results)

100%|██████████| 27385/27385 [37:02<00:00, 12.32it/s]


In [43]:
sat_recal=np.array(sat_recal)
mask_recal=np.array(mask_recal)

In [44]:
np.save('sat.npy',sat_recal)
np.save('mask.npy',mask_recal)

#### Conteo de clases

In [45]:
masks=[os.path.join(masks_path,i) for i in file_names]

In [16]:
mascaras=np.load('Input\mask.npy')
mascaras_conteo=np.where(mascaras != 0, 1, 0)
satelite=np.load('Input\sat.npy')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\gonza\AppData\Local\Temp\ipykernel_23168\3221419892.py:1: SyntaxWarning: invalid escape sequence '\m'
  mascaras=np.load('Input\mask.npy')
C:\Users\gonza\AppData\Local\Temp\ipykernel_23168\3221419892.py:3: SyntaxWarning: invalid escape sequence '\s'
  satelite=np.load('Input\sat.npy')


In [22]:
rescate_m=[]
rescate_s=[]
counters = {"t_n": 0, "m_n": 0, "n_n": 0}
for i,m in tqdm(enumerate(mascaras_conteo)):
    p= np.sum(m==0)/m.size
    if p == 1.0:
        counters["t_n"] += 1
    elif p > 0.6:
        counters["m_n"] += 1
    else:
        counters["n_n"] += 1
        rescate_m.append(mascaras[i])
        rescate_s.append(satelite[i])

27385it [00:02, 11731.62it/s]


In [14]:
print(f"""Negro Completo: {round(counters["t_n"] / 27385, 2)}
Mayormente negro: {round(counters["m_n"] / 27385, 2)}
Pog: {round(counters["n_n"] / 27385, 2)}
""")


Negro Completo: 0.12
Mayormente negro: 0.48
Pog: 0.4



In [24]:
np.save('f_sat.npy',np.array(rescate_s))
np.save('f_mask.npy',np.array(rescate_m))